In [1]:
!nvidia-smi

Wed Mar  4 17:40:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!wget -c "https://sutdapac-my.sharepoint.com/:u:/g/personal/gary_ong_mymail_sutd_edu_sg/EayDZjbTF3JAi1VTlyiU5skB5VLEdUjjFTrktsTm29WB_Q?e=aO1YJR&download=1" -O 701515.zip

--2020-03-04 17:40:41--  https://sutdapac-my.sharepoint.com/:u:/g/personal/gary_ong_mymail_sutd_edu_sg/EayDZjbTF3JAi1VTlyiU5skB5VLEdUjjFTrktsTm29WB_Q?e=aO1YJR&download=1
Resolving sutdapac-my.sharepoint.com (sutdapac-my.sharepoint.com)... 13.107.136.9
Connecting to sutdapac-my.sharepoint.com (sutdapac-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/gary_ong_mymail_sutd_edu_sg/Documents/BengaliDataset/701515.zip?&originalPath=aHR0cHM6Ly9zdXRkYXBhYy1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9nYXJ5X29uZ19teW1haWxfc3V0ZF9lZHVfc2cvRWF5RFpqYlRGM0pBaTFWVGx5aVU1c2tCNVZMRWRVampGVHJrdHNUbTI5V0JfUT9ydGltZT1jMVhUTEdQQTEwZw [following]
--2020-03-04 17:40:48--  https://sutdapac-my.sharepoint.com/personal/gary_ong_mymail_sutd_edu_sg/Documents/BengaliDataset/701515.zip?&originalPath=aHR0cHM6Ly9zdXRkYXBhYy1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9nYXJ5X29uZ19teW1haWxfc3V0ZF9lZHVfc2cvRWF5RFpqYlRGM0pBaTFWVGx5aVU1c2tCNVZMRWRVampG

In [3]:
!unzip 701515.zip

Archive:  701515.zip
  inflating: s_test_label2.csv       
  inflating: s_test2.npy             
  inflating: s_train_label2.csv      
  inflating: s_train2.npy            
  inflating: s_val_label2.csv        
  inflating: s_val2.npy              


In [4]:
!ls

701515.zip   s_test2.npy	s_train2.npy	    s_val2.npy
sample_data  s_test_label2.csv	s_train_label2.csv  s_val_label2.csv


In [5]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-ux8zm1yv
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-ux8zm1yv
     |████████████████████████████████| 634kB 5.1MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.4-cp36-none-any.whl size=64354 sha256=ce212f31f575ba27aabe76c968c9a63543162053ba2a03a6a177910bd54817e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-n1bt26ec/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=7611f7b5e93b60c064fdc6495ba1aa97e07ab034a3a7e3e73eeaf9acf323cc7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-n1bt26ec/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: album

In [0]:
from torch.utils.data import Dataset
import pandas as pd
import torch
import numpy as np
class BengaliDataset2(Dataset):
    def __init__(self,npy_file,label_csv,aug=None,norm=None):
        self.npy_file = np.load(npy_file)
        self.norm = norm
        df = pd.read_csv(label_csv)
        # for faster access i think
        self.grapheme_root = df["grapheme_root"].values
        self.vowel_diacritic = df["vowel_diacritic"].values
        self.consonant_diacritic = df["consonant_diacritic"].values

        self.aug = aug

    def __getitem__(self, index):
        image_arr = self.npy_file[index]
        # only do this on training
        #use albumentations library
        if self.aug != None:
            image_arr = self.aug(image=image_arr)["image"]

        image_arr = torch.from_numpy(image_arr)
        image_arr = image_arr/255.0 #normalize and converts to float32

        if self.norm != None:
            mean = self.norm['mean']
            std = self.norm['std']
            image_arr = (image_arr -  mean)/std

        grapheme_root = torch.Tensor([self.grapheme_root[index]]).long()
        vowel_diacritic = torch.Tensor([self.vowel_diacritic[index]]).long()
        consonant_diacritic = torch.Tensor([self.consonant_diacritic[index]]).long()
        
        return {"image":image_arr.unsqueeze(0),"grapheme_root":grapheme_root,"vowel_diacritic":vowel_diacritic,"consonant_diacritic":consonant_diacritic}

    def __len__(self):
        return self.npy_file.shape[0]

In [0]:
import albumentations as A
import torchvision.transforms as transforms
mean = 16.922
std = 51.398

shift_scale_rotate = A.augmentations.transforms.ShiftScaleRotate(p=0.5,scale_limit=0.4,rotate_limit=30)
brightness = A.augmentations.transforms.RandomBrightness(p=0.5)
# grid_distortion = A.augmentations.transforms.GridDistortion(p=0.5,distort_limit=0.4)
# blur = A.augmentations.transforms.Blur(p=0.2)
# opticalDist = A.augmentations.transforms.OpticalDistortion(p=0.5)
# elasticTransform = A.augmentations.transforms.ElasticTransform(p=0.5,alpha_affine=10)
# gd = A.augmentations.transforms.GridDropout(unit_size_min=20,unit_size_max=100,p=0.5)
aug_list = [shift_scale_rotate,brightness]
augment = A.core.composition.Compose(aug_list,p=0.5)
augment=None
train_data = BengaliDataset2("s_train2.npy","s_train_label2.csv",aug =augment,norm={'mean':mean,'std':std})
val_data = BengaliDataset2("s_val2.npy","s_val_label2.csv",norm={'mean':mean,'std':std})
test_data = BengaliDataset2("s_test2.npy","s_test_label2.csv",norm={'mean':mean,'std':std})

In [0]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
import matplotlib.pyplot as plt

def train_model(model, criterion, optimizer, device, dataloaders, scheduler=None, num_epochs=25):
    since = time.time()

    best_acc = 0.0

    dataset_sizes = {'train': len(dataloaders['train'].dataset),'val': len(dataloaders['val'].dataset),'test':len(dataloaders['test'].dataset)}

    train_acc_list = []; train_loss_list= []; val_acc_list = []; val_loss_list = []; test_acc_list = []; test_loss_list = []

    for epoch in range(num_epochs):
        start = time.time()
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val','test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            grapheme_corrects = 0
            vowel_corrects = 0
            consonant_corrects = 0


            # Iterate over data.
            for data in dataloaders[phase]:

                inputs = data['image']

                grapheme_root_label = data['grapheme_root']
                vowel_diacritic_label = data['vowel_diacritic']
                consonant_diacritic_label = data['consonant_diacritic']
                
         
                inputs = inputs.to(device)

                grapheme_root_label =  grapheme_root_label.to(device)
                vowel_diacritic_label = vowel_diacritic_label.to(device)
                consonant_diacritic_label =  consonant_diacritic_label.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    
                    grapheme_preds = outputs[:,:168].argmax(dim=1)
                    vowel_preds = outputs[:,168:179].argmax(dim=1) 
                    consonant_preds = outputs[:,179:186].argmax(dim=1)

                    loss = criterion(outputs, grapheme_root_label.squeeze(1),vowel_diacritic_label.squeeze(1),consonant_diacritic_label.squeeze(1))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                  
                # statistics
                running_loss += loss.item() * inputs.size(0)
                

                grapheme_corrects += torch.sum(grapheme_preds == grapheme_root_label.data.squeeze(1))
                vowel_corrects += torch.sum(vowel_preds == vowel_diacritic_label.data.squeeze(1))
                consonant_corrects += torch.sum(consonant_preds== consonant_diacritic_label.data.squeeze(1))
                
        

            epoch_loss = running_loss / dataset_sizes[phase]

            running_corrects = 0.5*grapheme_corrects.double() + 0.25*vowel_corrects.double() + 0.25*consonant_corrects.double()

            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            if phase == 'val' and scheduler != None:
                scheduler.step(epoch_loss)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == "train":
                # Note this are running values (calculated per batch) rather than actual values at the end of each epoch
                # Decreases training time
                # Not accurate especially at first few epochs
                train_acc_list.append(epoch_acc)
                train_loss_list.append(epoch_loss)
            elif phase == "val":
                val_acc_list.append(epoch_acc)
                val_loss_list.append(epoch_loss)
            elif phase == "test":
                test_acc_list.append(epoch_acc)
                test_loss_list.append(epoch_loss)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        end = time.time()
        print(f"time per epoch:{end-start}s")

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    plots = (train_acc_list,train_loss_list,val_acc_list,val_loss_list,test_acc_list,test_loss_list)

    return model, plots

def plot_model_metrics(plots,name):
    train_acc_list,train_loss_list,val_acc_list,val_loss_list,test_acc_list,test_loss_list = plots
    plot(train_acc_list,val_acc_list,test_acc_list,"accuracy",name)
    plot(train_loss_list,val_loss_list,test_loss_list,"loss",name)


def plot(train,val,test,metric,name):
    plt.title(name)
    plt.plot(train,label="train {}".format(metric))
    plt.plot(val,label="val {}".format(metric))
    plt.plot(test,label="test {}".format(metric))
    plt.legend(loc="best")
    plt.savefig("{}-{}".format(name,metric))
    plt.close()
    
def loss(outputs,grapheme_root_label,vowel_diacritic_label,consonant_diacritic_label):

    grapheme_root_output = outputs[:,:168]
    vowel_diacritic_output = outputs[:,168:179]
    consonant_diacritic_output = outputs[:,179:186]
    gloss = nn.CrossEntropyLoss()(grapheme_root_output,grapheme_root_label)
    vloss = nn.CrossEntropyLoss()(vowel_diacritic_output,vowel_diacritic_label)
    closs = nn.CrossEntropyLoss()(consonant_diacritic_output,consonant_diacritic_label)

    return 0.5*gloss + 0.25*vloss + 0.25*closs

def evaluate_test(model,criterion,dataloader,device):
    running_loss = 0.0
    grapheme_corrects = 0.0
    vowel_corrects = 0.0
    consonant_corrects = 0.0
    grapheme_output = []
    vowel_output = []
    
    for data in dataloader:

        inputs = data['image']

        grapheme_root_label = data['grapheme_root']
        vowel_diacritic_label = data['vowel_diacritic']
        consonant_diacritic_label = data['consonant_diacritic']

        inputs = inputs.to(device)

        grapheme_root_label =  grapheme_root_label.to(device)
        vowel_diacritic_label = vowel_diacritic_label.to(device)
        consonant_diacritic_label =  consonant_diacritic_label.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            
            grapheme_preds = outputs[:,:168].argmax(dim=1)
            vowel_preds = outputs[:,168:179].argmax(dim=1)
            consonant_preds = outputs[:,179:186].argmax(dim=1)

            loss = criterion(outputs, grapheme_root_label.squeeze(1),vowel_diacritic_label.squeeze(1),consonant_diacritic_label.squeeze(1))


        # statistics
        running_loss += loss.item() * inputs.size(0)
        

        grapheme_corrects += torch.sum(grapheme_preds == grapheme_root_label.data.squeeze(1))
        vowel_corrects += torch.sum(vowel_preds == vowel_diacritic_label.data.squeeze(1))
        consonant_corrects += torch.sum(consonant_preds== consonant_diacritic_label.data.squeeze(1))
        
    
    loss = running_loss / len(dataloader.dataset)

    running_corrects = 0.5*grapheme_corrects.double() + 0.25*vowel_corrects.double() + 0.25*consonant_corrects.double()

    # epoch_acc = running_corrects.double() / dataset_sizes[phase]
    acc = running_corrects / len(dataloader.dataset)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        "Final Test Accuracy", loss, acc))

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(train_data, batch_size=64, num_workers=4,shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, num_workers=4)
test_loader = DataLoader(test_data, batch_size=64, num_workers=4)

dataloaders = {'train': train_loader,'val': val_loader,'test':test_loader}


In [77]:
# train all layers with pretrained model
model = models.resnet18(pretrained=True)
cn1 = nn.Parameter(torch.mean(model.conv1.weight, dim=1, keepdim=True))
print(cn1[0])
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
model.conv1.weight = cn1

tensor([[[-0.0079, -0.0140, -0.0051,  0.0671,  0.0594,  0.0171, -0.0195],
         [ 0.0241,  0.0082, -0.1135, -0.2950, -0.2791, -0.1398, -0.0085],
         [ 0.0007,  0.0689,  0.3049,  0.5959,  0.5254,  0.2432,  0.0688],
         [ 0.0170, -0.0739, -0.3054, -0.4331, -0.2530,  0.0119,  0.0800],
         [-0.0373,  0.0097,  0.0661, -0.0481, -0.3040, -0.4163, -0.2593],
         [ 0.0281,  0.0313,  0.0416,  0.2049,  0.4005,  0.3733,  0.1609],
         [-0.0026,  0.0013, -0.0171, -0.0636, -0.1492, -0.0923, -0.0145]]],
       grad_fn=<SelectBackward>)


In [78]:
print(model.conv1.weight[0])

tensor([[[-0.0079, -0.0140, -0.0051,  0.0671,  0.0594,  0.0171, -0.0195],
         [ 0.0241,  0.0082, -0.1135, -0.2950, -0.2791, -0.1398, -0.0085],
         [ 0.0007,  0.0689,  0.3049,  0.5959,  0.5254,  0.2432,  0.0688],
         [ 0.0170, -0.0739, -0.3054, -0.4331, -0.2530,  0.0119,  0.0800],
         [-0.0373,  0.0097,  0.0661, -0.0481, -0.3040, -0.4163, -0.2593],
         [ 0.0281,  0.0313,  0.0416,  0.2049,  0.4005,  0.3733,  0.1609],
         [-0.0026,  0.0013, -0.0171, -0.0636, -0.1492, -0.0923, -0.0145]]],
       grad_fn=<SelectBackward>)


In [0]:


model = model.to(device)
criterion = loss
optimizer = optim.Adam(model.parameters(),lr=1e-4)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',patience=5,factor=0.3,verbose=True)

model,plots = train_model(model, criterion, optimizer,
            device, dataloaders,scheduler=scheduler, num_epochs=30)

evaluate_test(model,criterion,test_loader,device)
# plot_model_metrics(plots,"graph")


print("done")

Epoch 1/30
----------
train Loss: 0.7543 Acc: 0.7998
val Loss: 0.3195 Acc: 0.9075
test Loss: 0.3248 Acc: 0.9061
time per epoch:122.94647932052612s
Epoch 2/30
----------
train Loss: 0.2378 Acc: 0.9303
val Loss: 0.2560 Acc: 0.9252
test Loss: 0.2624 Acc: 0.9245
time per epoch:123.3808844089508s
Epoch 3/30
----------
train Loss: 0.1572 Acc: 0.9531
val Loss: 0.3322 Acc: 0.8976
test Loss: 0.3421 Acc: 0.8951
time per epoch:122.63356041908264s
Epoch 4/30
----------


In [0]:
torch.Tensor([[1,2],[3,4]]).unsqueeze(0).shape